In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, GPT CBETA DC GUN example

This is the class method for running Xopt.

In [ ]:
from xopt import Xopt

# Notebook printing output
from xopt import output_notebook
output_notebook()

In [ ]:
import yaml

# Make a proper input file. 

YAML="""
xopt:
    output_path: null
  
algorithm:
  name: cnsga
  options: {max_generations: 16, population_size: 24, crossover_probability: 0.9, mutation_probability: 1.0,
  selection: auto, show_progress: True, population: null}

simulation:
  name: gpt_with_distgen
  function: gpt.gpt_distgen.evaluate_gpt_with_distgen
  options:
    gpt_input_file: templates/cbeta_dcgun/cbeta_dcgun_gpt.in
    distgen_input_file: templates/cbeta_dcgun/cbeta_dcgun_distgen.yaml
      

# Note that you can call another file in the top level group:
vocs: templates/cbeta_dcgun/cbeta_dcgun_vocs.yaml

"""
config = yaml.safe_load(YAML)

In [ ]:
# Create object
X = Xopt(config=config)

# Show config
X

In [ ]:
# Check vocs
X.vocs

In [ ]:
# Check random inputs
X.random_inputs()

In [ ]:
?X.random_evaluate

In [ ]:
%%time
# Do a random evaluate to check that everything will run
x=X.random_evaluate(check_vocs=True)
print(x)

In [ ]:
X.vocs['constraints']

In [ ]:
# These are the algorithm options
X.algorithm['options']

In [ ]:
# These are the options in the evaluate function
X.simulation['options']

# Run CNSGA using processes or threads

In [ ]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [ ]:
X.run(executor=executor)

# Ouput
The last generation, as well as important simulation data is saved in xopt.population:

In [ ]:
print('Available population data:',list(X.results.keys()))
print('See which individuals had errors:', X.results['error'])

In [ ]:
from matplotlib import pyplot as plt
import matplotlib 
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

qs =  np.array([indin['total_charge:value'] for indin in X.results['inputs']])
ens = np.array([indout['end_norm_emit_x'] for indout in X.results['outputs']])*1e6

plt.plot(qs, ens, '*')
plt.xlabel('$q_b$ (nC)')
plt.ylabel('$\epsilon_{n,x}$ ($\mu$m)');

# Write this configuration

In [ ]:
X.save('test.yaml')

In [ ]:
!cat test.yaml

# Run with MPI

In [ ]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run test.yaml

# Dask 

In [ ]:
from dask.distributed import Client
executor = Client()
executor

In [ ]:
# Wait a few seconds for the Dask cluster to start
from time import sleep
sleep(5)

In [ ]:
# Increase the number of generations
X.algorithm['options']['max_generations'] = 7
X.run(executor=executor)

In [ ]:
X.population.keys()

# Cleanup

In [ ]:
!rm -r dask-worker-space

In [ ]:
!rm test.yaml